In [ ]:
!pip install --upgrade wfdb
!pip install tensorflow_model_optimization
!pip install --upgrade kerop
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import wfdb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from matplotlib.legend_handler import HandlerLine2D
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/MyDrive/mit-bih-arrhythmia-database-1.0.0.zip

In [ ]:
#records = np.loadtxt("/content/gdrive/MyDrive/Datasets/RHMS/ARR/RECORDS", dtype=int)
records = np.loadtxt("mit-bih-arrhythmia-database-1.0.0/RECORDS", dtype=int)

In [ ]:
#These are the beat classifications according to physiobank

invalid = [ "[", "!", "]", "x", "(", ")", "p", "t", "u", "`", "'", "^", "|", "~", "+", "s", "T", "*", "D", "=", '"', "@" ]

abnormal = [ "L", "R", "B", "A", "a", "J", "S", "V", "r", "F", "e", "j", "n", "E", "/", "f", "Q", "?" ]

In [ ]:
def classify_beat(symbol):
    if symbol in abnormal :
        return 1
    elif symbol == "N" or symbol == ".":
        return 0

In [ ]:
def get_sequence(signal, beat_loc, window_sec, fs):
    window_one_side = window_sec * fs
    beat_start = beat_loc - window_one_side
    beat_end = beat_loc + window_one_side
    if beat_end < signal.shape[0]:
        sequence = signal[beat_start:beat_end, 0]
        return sequence.reshape(1, -1, 1)
    else:
        return np.array([])

In [ ]:
all_sequences = []
all_labels = []
window_sec = 3
subject_map = []
for subject in records:
    #record = wfdb.rdrecord(f'/content/gdrive/MyDrive/Datasets/RHMS/ARR/{subject}')
    #annotation = wfdb.rdann(f'/content/gdrive/MyDrive/Datasets/RHMS/ARR/{subject}', 'atr')
    record = wfdb.rdrecord(f'mit-bih-arrhythmia-database-1.0.0/{subject}')
    annotation = wfdb.rdann(f'mit-bih-arrhythmia-database-1.0.0/{subject}', 'atr')
    atr_symbol = annotation.symbol
    atr_sample = annotation.sample
    fs = record.fs
    scaler = StandardScaler()
    signal = scaler.fit_transform(record.p_signal)
    subject_labels = []
    for i, i_sample in enumerate(atr_sample):
        label = classify_beat(atr_symbol[i])
        sequence = get_sequence(signal, i_sample, window_sec, fs)
        if label is not None and sequence.size > 0:
            all_sequences.append(sequence)
            subject_labels.append(label)

    normal_percentage = sum(subject_labels) / len(subject_labels)
    subject_map.append({
        "subject": subject,
        "percentage": normal_percentage,
        "num_seq": len(subject_labels),
        "start": len(all_labels),
        "end": len(all_labels)+len(subject_labels)
    })
    all_labels.extend(subject_labels)

In [ ]:
subject_map = pd.DataFrame(subject_map)

In [ ]:
bins = [0, 0.2, 0.6, 1.0]
subject_map["bin"] = pd.cut(subject_map['percentage'], bins=bins, labels=False, include_lowest=True)

In [ ]:
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [ ]:
def build_dataset(df, all_sequences, all_labels):
    sequences = []
    labels = []
    for i, row in df.iterrows():
        start = int(row["start"])
        end = int(row["end"])
        sequences.extend(all_sequences[start:end])
        labels.extend(all_labels[start:end])
        
    return np.vstack(sequences), np.vstack(labels)

In [ ]:
#train, validation = train_test_split(subject_map, test_size=0.25, stratify=subject_map["bin"], random_state=42)
train, validation, test = train_validate_test_split(subject_map)
train, validation, test

In [ ]:
X_train, y_train = build_dataset(train, all_sequences, all_labels)
X_val, y_val = build_dataset(validation, all_sequences, all_labels)
X_test, y_test = build_dataset(test, all_sequences, all_labels)

In [ ]:
data = pd.DataFrame(X_val.reshape(X_val.shape[0], X_val.shape[1]))
data['label'] = y_val
# This conversion takes a few minutes
data.to_csv('/content/gdrive/MyDrive/val.csv')

In [ ]:
data

In [ ]:
X_train.shape, y_train.shape

((63728, 2160, 1), (63728, 1))

In [ ]:
X_val.shape, y_val.shape

((19968, 2160, 1), (19968, 1))

In [ ]:
X_test.shape, y_test.shape

((25436, 2160, 1), (25436, 1))

In [ ]:
X_train1 = np.reshape(X_train,(X_train.shape[0],X_train.shape[1]))
X_val1 = np.reshape(X_val,(X_val.shape[0],X_val.shape[1]))
X_test1 = np.reshape(X_test,(X_test.shape[0], X_test.shape[1]))

The Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=10,
                            random_state=0)
rf.fit(X_train1, y_train.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = rf.predict(X_test1)
y_pred

array([1, 1, 0, ..., 0, 0, 0])

In [ ]:
# The training accuracy of the Random forest
rf.score(X_train1,y_train)

0.998054230479538

In [ ]:
# The test accuracy of the Random forest
rf.score(X_test1,y_test)

0.8395581066205379

In [ ]:
# The final validation accuracy of the Random forest
rf.score(X_val1,y_val)

0.918770032051282

In [ ]:
# Execute this later
# print("Classification report - \n", classification_report(y_test,y_pred))

NameError: ignored

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.7602019856393482

Testing:


In [ ]:
# Trying to estimate the optimal value of n in the RF
# Takes approx. 20 years (jk, around 35 mins.) to execute, for n = 100 and n = 200. Do NOT execute this more than once. Check graph for the optimal value of n
n_estimators = [1, 2, 4, 8, 10, 16, 32, 64, 100, 200]
train_results = []
test_results = []
for estimator in n_estimators:
   rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)
   rf.fit(X_train1, y_train)   
   train_pred = rf.predict(X_train1)   
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   train_results.append(roc_auc)   
   y_pred = rf.predict(X_test1)   
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   test_results.append(roc_auc)


In [ ]:
line1 = plt.plot(n_estimators, train_results, 'b', label="Train AUC")
line2 = plt.plot(n_estimators, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('n_estimators')
plt.show()

In [ ]:
# Take 2 on trying to find the proper hyperparameters

def plot_roc_curve(fprs, tprs):
    """Plot the Receiver Operating Characteristic from a list
    of true positive rates and false positive rates."""
    
    # Initialize useful lists + the plot axes.
    tprs_interp = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    f, ax = plt.subplots(figsize=(14,10))
    
    # Plot ROC for each K-Fold + compute AUC scores.
    for i, (fpr, tpr) in enumerate(zip(fprs, tprs)):
        tprs_interp.append(np.interp(mean_fpr, fpr, tpr))
        tprs_interp[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        ax.plot(fpr, tpr, lw=1, alpha=0.3,
                 label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
        
    # Plot the luck line.
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
             label='Luck', alpha=.8)
    
    # Plot the mean ROC.
    mean_tpr = np.mean(tprs_interp, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(mean_fpr, mean_tpr, color='b',
             label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
             lw=2, alpha=.8)
    
    # Plot the standard deviation around the mean ROC.
    std_tpr = np.std(tprs_interp, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                     label=r'$\pm$ 1 std. dev.')
    
    # Fine tune and show the plot.
    ax.set_xlim([-0.05, 1.05])
    ax.set_ylim([-0.05, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic')
    ax.legend(loc="lower right")
    plt.show()
    return (f, ax)

def compute_roc_auc(index):
    y_predict = rf.predict_proba(X_train1[index])[:,1]
    fpr, tpr, thresholds = roc_curve(y_train[index], y_predict)
    auc_score = auc(fpr, tpr)
    return fpr, tpr, auc_score

cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
results = pd.DataFrame(columns=['training_score', 'test_score'])
fprs, tprs, scores = [], [], []
    
for (train, test), i in zip(cv.split(X_train1, y_train), range(5)):
    rf.fit(X_train1[train], y_train[train])
    _, _, auc_score_train = compute_roc_auc(train)
    fpr, tpr, auc_score = compute_roc_auc(test)
    scores.append((auc_score_train, auc_score))
    fprs.append(fpr)
    tprs.append(tpr)

plot_roc_curve(fprs, tprs);
pd.DataFrame(scores, columns=['AUC Train', 'AUC Test'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: DataConversionWarning: A column-vector y was passed whe

ImportError: ignored

<Figure size 1008x720 with 1 Axes>

,AUC Train,AUC Test
0,0.999994,0.990359
1,0.999991,0.991889
2,0.999989,0.991940
3,0.999994,0.992259
4,0.999991,0.990364


In [ ]:
roc_auc_score(y_test, y_pred, average='macro', sample_weight=None, max_fpr=None, multi_class='raise', labels=None)

In [ ]:
fig = plt.figure(figsize=(25,20))
#fn=data.feature_names
#cn=data.target_names
tree.plot_tree(rf.estimators_[0],
               filled = True);
fig.savefig('rf_full.png')